In [0]:

data=[['Brampton','Miss','Tor'],['', 'Toronto',''],['','','mississauga']]
columns=['city1','city2','city3']
df = spark.createDataFrame(data,columns)
from pyspark.sql.functions import *
df1= df.withColumn('city', coalesce(
                    when(df.city1 =="",None).otherwise(df.city1),
                    when(df.city2 =="",None).otherwise(df.city2),
                    when(df.city3 =="",None ).otherwise(df.city3)
                                    ))
display(df1)


In [0]:
data1=[(1,"Steve"),(2,"David"),(3,"John"),(4,"Shree"),(5,"Helen")]
data2=[(1,"SQL",90),(1,"PySpark",100),(2,"SQL",70),(2,"PySpark",60),(3,"SQL",30),(3,"PySpark",20),(4,"SQL",50),(4,"PySpark",50),(5,"SQL",45),(5,"PySpark",45)]

schema1=["Id","Name"]
schema2=["Id","Subject","Mark"]

df1=spark.createDataFrame(data1,schema1)
df2=spark.createDataFrame(data2,schema2)
df_ = df1.join(df2,df1.Id==df2.Id,how='inner').drop(df1.Id)


In [0]:
df_agg = df_.groupBy('id','Name').agg((sum("Mark")/count("Mark")).alias("percentage"))
#display(df_agg)
df_fin = df_agg.select('id','Name','percentage',
              when(df_agg.percentage>80,"A")
             .when(df_agg.percentage>60,"B")
             .when(df_agg.percentage>40,"C")
             .when(df_agg.percentage>20,"D")
             .otherwise("F")
             .alias("grade"))
display(df_fin)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

data1=[(1,"A",1000,"IT"),(2,"B",1500,"IT"),(3,"C",2500,"IT"),(4,"D",3000,"HR"),(5,"E",2000,"HR"),(6,"F",1000,"HR")
       ,(7,"G",4000,"Sales"),(8,"H",4000,"Sales"),(9,"I",1000,"Sales"),(10,"J",2000,"Sales")]
schema1=["EmpId","EmpName","Salary","DeptName"]
df=spark.createDataFrame(data1,schema1)

df_rank = df.withColumn("rank",dense_rank().over(Window.partitionBy("DeptName").orderBy(desc("Salary"))))
df_rank1 = df.select("*", (dense_rank().over(Window.partitionBy(df.DeptName).orderBy(df.Salary.desc()))).alias("rank"))
display(df_rank1.filter(df_rank1.rank==2))

In [0]:
#df.rdd.getNumPartition()
df.repartition(10)
from pyspark.sql.functions import spark_partition_id
df.select(spark_partition_id().alias("partition")).groupBy("partition").count().show()




In [0]:
# (modifiedBefore & modifiedAfter) file properties